In [7]:
%run ../spark-default.py

In [8]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [9]:
import builtins as b
from datetime import datetime, timedelta, date
import sys, traceback
from delta.tables import DeltaTable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# spark.sql("optimize stage.ciha_step1")

In [11]:
# Até aqui, a base do CIHA foi enriquecida com:
#  - Nome das cidades (residencia e movimentação)
#  - Nome, uf e info dos estados (residencia e movimentação)
#  - Infos do IBGE como população e área
#  - Calculo de feriados

In [12]:
ciha_step2 = spark.table("stage.ciha_step2")
ciha_step2.printSchema()

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [20]:
ciha_step2.select([count(when(col(c).isNull(), c)).alias(c) for c in ciha_step2.columns]).show()

+--------+---------+--------+---------+----+----+----------+----------+--------+-------+--------+------------+--------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+---------+---------+---------+----+-----+----------+----------+---------------+----------------+--------+-----------------+-----------------+------------------------+------------------------+------------------+------------------+----------------+----------------+-----------------------------+-----------------------------+----------------------------------+----------------------------------+-------+----------------+----------------+
|mes_cmpt|    espec|cgc_hosp|munic_res|nasc|sexo|uti_mes_to|uti_int_to|proc_rea|qt_proc|dt_atend|dt_atend_dow|dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte| nacional|  car_int| homonimo|cnes|fonte|cgc_consor|modalidade|input_file_name|arquivo_sigla_uf|ano_cmpt|munic_res_nome_uf|munic_mov_nome_uf|munic_r

In [22]:
ciha_step3 = ciha_step2 \
    .select(
        col("mes_cmpt"),
        col("espec"),
        col("cgc_hosp"),
        col("munic_res"),
        col("nasc"),
        col("sexo"),
        col("uti_mes_to"),
        col("uti_int_to"),
        col("proc_rea"),
        col("qt_proc"),
        col("dt_atend"),
        col("dt_atend_dow"),
        col("dt_saida"),
        col("diag_princ"),
        col("diag_secun"),
        col("cobranca"),
        col("natureza"),
        col("gestao"),
        col("munic_mov"),
        col("cod_idade"),
        col("idade"),
        col("dias_perm"),
        col("morte"),
        col("nacional"),
        col("car_int"),
        col("homonimo"),
        col("cnes"),
        col("fonte"),
        col("cgc_consor"),
        col("modalidade"),
        col("input_file_name"),
        col("arquivo_sigla_uf"),
        col("ano_cmpt"),
        col("munic_res_nome_uf"),
        col("munic_mov_nome_uf"),
        col("munic_res_nome_municipio"),
        col("munic_mov_nome_municipio"),
        col("munic_res_sigla_uf"),
        col("munic_mov_sigla_uf"),
        col("munic_res_regiao"),
        col("munic_mov_regiao"),
        col("munic_res_populacao_residente"),
        col("munic_mov_populacao_residente"),
        col("munic_res_area_unidade_territorial"),
        col("munic_mov_area_unidade_territorial"),
        
        col("feriado.feriado"),
        col("feriado.distancia_feriado").alias("feriado_distancia"),
        col("feriado.feriado_info"),

        col("diag_princ_cid10.sub_categoria").alias("diag_princ_sub_categoria"),
        col("diag_princ_cid10.sub_categoria_classificacao").alias("diag_princ_sub_categoria_classificacao"),
        col("diag_princ_cid10.restringido_sexo").alias("diag_princ_restringido_sexo"),
        col("diag_princ_cid10.causa_obito").alias("diag_princ_causa_obito"),
        col("diag_princ_cid10.sub_categoria_descricao").alias("diag_princ_sub_categoria_descricao"),
        col("diag_princ_cid10.sub_categoria_descricao_abreviada").alias("diag_princ_sub_categoria_descricao_abreviada"),
        col("diag_princ_cid10.sub_categoria_referencia").alias("diag_princ_sub_categoria_referencia"),
        col("diag_princ_cid10.sub_categoria_excluidos").alias("diag_princ_sub_categoria_excluidos"),
        col("diag_princ_cid10.categoria").alias("diag_princ_categoria"),
        col("diag_princ_cid10.categoria_classificacao").alias("diag_princ_categoria_classificacao"),
        col("diag_princ_cid10.categoria_descricao").alias("diag_princ_categoria_descricao"),
        col("diag_princ_cid10.categoria_descricao_abreviada").alias("diag_princ_categoria_descricao_abreviada"),
        col("diag_princ_cid10.categoria_referencia").alias("diag_princ_categoria_referencia"),
        col("diag_princ_cid10.capitulo_numero").alias("diag_princ_capitulo_numero"),
        col("diag_princ_cid10.capitulo_descricao").alias("diag_princ_capitulo_descricao"),
        col("diag_princ_cid10.capitulo_descricao_abreviada").alias("diag_princ_capitulo_descricao_abreviada"),

        col("diag_secun_cid10.sub_categoria").alias("diag_secun_sub_categoria"),
        col("diag_secun_cid10.sub_categoria_classificacao").alias("diag_secun_sub_categoria_classificacao"),
        col("diag_secun_cid10.restringido_sexo").alias("diag_secun_restringido_sexo"),
        col("diag_secun_cid10.causa_obito").alias("diag_secun_causa_obito"),
        col("diag_secun_cid10.sub_categoria_descricao").alias("diag_secun_sub_categoria_descricao"),
        col("diag_secun_cid10.sub_categoria_descricao_abreviada").alias("diag_secun_sub_categoria_descricao_abreviada"),
        col("diag_secun_cid10.sub_categoria_referencia").alias("diag_secun_sub_categoria_referencia"),
        col("diag_secun_cid10.sub_categoria_excluidos").alias("diag_secun_sub_categoria_excluidos"),
        col("diag_secun_cid10.categoria").alias("diag_secun_categoria"),
        col("diag_secun_cid10.categoria_classificacao").alias("diag_secun_categoria_classificacao"),
        col("diag_secun_cid10.categoria_descricao").alias("diag_secun_categoria_descricao"),
        col("diag_secun_cid10.categoria_descricao_abreviada").alias("diag_secun_categoria_descricao_abreviada"),
        col("diag_secun_cid10.categoria_referencia").alias("diag_secun_categoria_referencia"),
        col("diag_secun_cid10.capitulo_numero").alias("diag_secun_capitulo_numero"),
        col("diag_secun_cid10.capitulo_descricao").alias("diag_secun_capitulo_descricao"),
        col("diag_secun_cid10.capitulo_descricao_abreviada").alias("diag_secun_capitulo_descricao_abreviada")
    )

    
ciha_step3.printSchema()
ciha_step3.show(truncate=False)

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [32]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.ciha_step2") \
  .addColumns(ciha_step2.schema) \
  .partitionedBy("ano_cmpt") \
  .execute()

In [ ]:
ciha_step2.write.insertInto("stage.ciha_step2")

In [ ]:
final_sample = (final
     .withColumn("diag_princ_detalhes", to_json("diag_princ_detalhes"))
     .withColumn("diag_secun_detalhes", to_json("diag_secun_detalhes"))
     .withColumn("feriado_info", to_json("feriado_info"))
     .sample(fraction=0.001, seed = 1010)
)

(final_sample.repartition(1)
     .write
     .mode("overwrite")
     .option("header", True)
     .option("sep", ";")
     .csv("bases finais/final_sample/")
)

In [ ]:
final_sample.columns

In [ ]:
# To do
#   Add mesoregiao
#   Add dia da semana
#   

In [6]:
spark.stop()